# Table Storage Model




In [ ]:
!pip install sqlalchemy psycopg2-binary ipython-sql
!sudo apt-get install postgresql-client

import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

DB_USER   = "gpadmin"
DB_PWD    = "TRWLO7VX3SHLX"
DB_SERVER = "34.87.155.108"
DB_PORT   = "5432"
DB_NAME   = "dev"

CONNECTION_STRING=f"postgresql://{DB_USER}:{DB_PWD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}"

%reload_ext sql
%sql $CONNECTION_STRING

In [ ]:
query = "SHOW gp_autostats_mode; \
ALTER DATABASE {} SET gp_autostats_mode TO 'NONE'; \
SHOW gp_autostats_mode;".format(DB_NAME)

%sql $DB_USER@$DB_NAME {''.join(query)}

In [ ]:
query = !cat script/7-db-maintenance.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

## 7. Comparing Table Storage Models

Re-create the Amazon Reviews table, using 2 different table storage models, row-oriented and column-oriented, as shown below:

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-ro.sql
sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-co.sql

display_html('\n'.join(sqlfilecode1), raw=True)
display_html('\n'.join(sqlfilecode3), raw=True)

query1 = !cat script/7-1-amzn-reviews-ro.sql
query3 = !cat script/7-1-amzn-reviews-co.sql

%sql $DB_USER@$DB_SERVER {''.join(query1)}
%sql $DB_USER@$DB_SERVER {''.join(query3)}

### 7.1 Loading

#### 7.1.1 Loading from another source table

Load the two tables using table `demo.amzn_reviews` as source (see [Notebook 1](AWS-GP-demo-1.ipynb)), and compare their loading times.

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-load-amzn-reviews-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-1-load-amzn-reviews-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode2 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-load-amzn-reviews-co.sql
display_html('\n'.join(sqlfilecode2), raw=True)
cmd2 = !echo $(cat script/7-1-load-amzn-reviews-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd2), raw=True)

#### 7.1.2 Loading from a source file (Bulk Loading)

Let's drop & recreate the tables, load the input dataset in bulk to each using the `gpload` utility, and compare their loading times.

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-ro.sql
sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-co.sql

display_html('\n'.join(sqlfilecode1), raw=True)
display_html('\n'.join(sqlfilecode3), raw=True)

query1 = !cat script/7-1-amzn-reviews-ro.sql
query3 = !cat script/7-1-amzn-reviews-co.sql

%sql $DB_USER@$DB_SERVER {''.join(query1)}
%sql $DB_USER@$DB_SERVER {''.join(query3)}

In [ ]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload-amzn-reviews-ro.log ]; then rm ./gpload-amzn-reviews-ro.log; fi'
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-ro.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-ro.yaml
cmd = "gpload -d {0} -f ./gpload-amzn-reviews-ro.yaml -l ./gpload-amzn-reviews-ro.log 2>&1".format(DB_NAME) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

In [ ]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload-amzn-reviews-co.log ]; then rm ./gpload-amzn-reviews-co.log; fi'
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-co.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-co.yaml
cmd = "gpload -d {0} -f ./gpload-amzn-reviews-co.yaml -l ./gpload-amzn-reviews-co.log 2>&1".format(DB_NAME) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

In [ ]:
cmd = 'grep -e '"'"'running'"'"' /home/gpadmin/gpload-amzn-reviews*\
    | awk '"'"'BEGIN{FS=":"} {print $1, "finished in", $5}'"'"'' 
grep_output = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd | pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(grep_output), raw=True)

### 7.2 Table Size and Disk Space Usage

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/7-2-table-size-comparison.sql
display_html('\n'.join(sqlfilecode), raw=True)
query = !cat script/7-2-table-size-comparison.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

### 7.3 Query Performance

#### 7.3.0 `ANALYZE` tables

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-analyze.sql
display_html('\n'.join(sqlfilecode), raw=True)
query = !cat script/7-3-analyze.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

#### 7.3.1 Narrow (*Few columns of the table*) `SELECT`

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-narrow-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-narrow-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

#### 7.3.2.1 Super Narrow (*1 column of the table*) `SELECT`: "Short" Data Field example

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-super-narrow-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-super-narrow-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

#### 7.3.2.2 Super Narrow (*1 column of the table*) `SELECT`:  "Long" Data Field example

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-ro-2.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-super-narrow-select-ro-2.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-co-2.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-super-narrow-select-co-2.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

#### 7.3.3 Wide (*Most/Many columns of the table*) `SELECT`

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-wide-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-wide-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

#### 7.3.4.1 Aggregate/Window Functions over a limited number of columns

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-aggr-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-aggr-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

#### 7.3.4.2 Aggregate/Window Functions over a more columns

In [ ]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-ro-2.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-aggr-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-co-2.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-aggr-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)